# 實作 - 寫入檔案
# 萃取字詞的四大方法
# 首先要先確定有 [字典] 和 [停用詞] 的txt檔

In [1]:
import os
import jieba
import jieba.analyse
import jieba.posseg as pseg  
jieba.set_dictionary('dict.big.tra.txt')  #預設字典
jieba.load_userdict('dict_twstd_tfidf.txt')  #中文分詞詞庫(TFIDF)
jieba.load_userdict('dict_ntusd_pos.txt') #NTUSD_負向
jieba.load_userdict('dict_ntusd_nag.txt') #NTUSD_正向
jieba.load_userdict('dict_twedu_dict.txt') #教育部《重編國語辭典 修訂本》


#jieba.load_userdict('dict_20160111_NotReallyUseful.txt')  
#中文分詞詞庫(TFIDF) + 百度分詞詞庫 + 69萬中文大辭典 + 中文热门词库(155073) 
#不是很好用，可備著使用


jieba.analyse.set_stop_words("stopwords_1227.txt") #停用詞

Building prefix dict from C:\Users\BigData\dict.big.tra.txt ...
DEBUG:jieba:Building prefix dict from C:\Users\BigData\dict.big.tra.txt ...
Loading model from cache c:\users\bigdata\appdata\local\temp\jieba.ud8ec30fabaf60e161f06b3552aab2f0e.cache
DEBUG:jieba:Loading model from cache c:\users\bigdata\appdata\local\temp\jieba.ud8ec30fabaf60e161f06b3552aab2f0e.cache
Loading model cost 0.263 seconds.
DEBUG:jieba:Loading model cost 0.263 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


In [2]:
# 單一景點使用[Jieba TFIDF]的方法
def jiebastat(attraction,rankNum,filename):
#轉換象山資料,並切詞,並使用JIEBA內建的tfidf統計排名
    f = open(attraction,'r')  #讀取檔案
    f1 = f.read()  #接住
    f2 = ''.join(f1.split())  #把空白全去掉
    #print f2
    words=jieba.cut(f2)  #斷詞
    n = ' '.join(words)  #把斷詞結果用空白隔開
    n1 = n.encode('utf-8')  #編碼
    f.close()


    temp = []
    temp2 =[]
    x = jieba.analyse.extract_tags(n1, topK=rankNum, withWeight=True)
    for u in x:
        #print u[0],"\t",u[1]
        temp.append(u[0].encode('utf-8'))  #存詞彙
        temp2.append(u[1])  #存權重
        
    fid = open(filename,'a')
    #統計字頻
    dic={}
    for w in n1.split(' '):
        if w not in dic:
            dic[w] = 1
        else:
            dic[w] = dic[w] + 1
    import operator
    words_freq = sorted(dic.iteritems(),key=operator.itemgetter(1),reverse=True)
    #print attraction#,"\n總詞量：",len(words_freq)
    fid.write(attraction+',')
    fid.write("\n")
    

    #根據Jieba tf-idf去看字頻排名
    #print "=========字頻統計(根據jieba TFIDF)========="
    #print "排名","\t","總排","\t","詞彙","\t","出現次數","  ","權重","\t\t\t","比率"
    #print "- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -"
    num = 0
    num3 = -1
    for ele in words_freq:
        num += 1
        if ele[0] in temp:
            num3 += 1
            #print num3+1,"\t",num,"\t",ele[0],"\t",ele[1],"\t","   ",temp2[num3],\
            #"\t",float(int(( float(ele[1]) / float(len(words_freq)) ) *10000))/100,'%'
            #print ele[0]
            fid.write(ele[0]+',\n')
    fid.write("\n")
    fid.close()
# 比對多個景點[Jieba TFIDF]的方法
def mutijiebastat(date,attrlist,RankNum):
    filename = 'JiebaTfidf-'+str(RankNum)+'-'+date+'.txt'    
    for i in range(0,len(attrlist)):
        jiebastat(attrlist[i],RankNum,filename)
        #print '\n'
########       
# 方法二：計算Scikit-Learn的TFIDF值
########
def wordsToList(attfile):
    f = open(attfile,'r')
    f1 = f.read()
    f2 = ''.join(f1.split())
    #print f2
    words=jieba.cut(f2)
    n = ' '.join(words)
    n1 = n.encode('utf-8')
    return n1
    f.close()

def stopWordsList(stopwordfile):
    li = []
    fid =  open(stopwordfile,'r')
    for line in fid:
        li.append(line.strip().decode('utf-8'))#要解開utf-8,才能成功給scikitlearn用
    fid.close()
    return li

def CompareTFIDF(date,listInput,stopWordsFile,rankNum):
    from sklearn import feature_extraction  
    from sklearn.feature_extraction.text import TfidfTransformer  
    from sklearn.feature_extraction.text import CountVectorizer
    filename = 'SciTfidf-'+str(rankNum)+'-'+date+'.txt'
    fid = open(filename,'a')
    #創造一個空矩陣
    corpus = []
    
    #把傳入的List依序讀出，使用方法把這些東西全都讀出來
    for each in range(0,len(listInput)):
        corpus.append(wordsToList(listInput[each]))

    vectorizer=CountVectorizer(analyzer='word',stop_words=stopWordsList(stopWordsFile))  #該類會將文本中的詞語轉換為詞頻矩陣，矩陣元素a[i][j] 表示j詞在i類文本下的詞頻  
    x = vectorizer.fit_transform(corpus)  #將文本轉為詞頻矩陣 
    transformer=TfidfTransformer()  #該類會統計每個詞語的tf-idf權值 
    tfidf=transformer.fit_transform(x)  #計算tf-idf
    word=vectorizer.get_feature_names()  #獲取詞袋模型中的所有詞語
    weight=tfidf.toarray()   #將tf-idf矩陣抽取出來，元素a[i][j]表示j詞在i類文本中的tf-idf權重  

    #print '-------以下輸出各景點中TFIDF值最高的幾個詞語------\n'
    
    #打印每類文本的tf-idf詞語權重，第一個for遍歷所有文本，第二個for便利某一類文本下的詞語權重
    import operator
    for i in range(len(weight)):
        #print listInput[i]
        fid.write(listInput[i]+',')
        fid.write('\n')
        #print listInput[i],"\n------文章內詞語tf-idf權重------" 
        #print "排名",'\t',"詞語","\t","權重"
        #print "- - - - - - - - - - - - - - - -"
        dic = {}
        for j in range(len(word)):  
            dic[word[j]] = weight[i][j]
        words_freq = sorted(dic.iteritems(),key=operator.itemgetter(1),reverse=True)
        num2 = 0
        for ele in words_freq[0:rankNum]:
            num2 += 1
            #print num2,"\t",ele[0],"\t",ele[1]
            #print ele[0]
            fid.write(ele[0].encode('utf-8')+',')
            fid.write("\n")
        #print '\n'
        fid.write('\n')
    fid.close()
########       
# 方法三：計算字頻 - 文章斷詞的原始字頻
########
def jiebafrestat(attraction,rankNum,filename):
#轉換象山資料,並切詞,並使用JIEBA內建的tfidf統計排名
    f = open(attraction,'r')  #讀取檔案
    f1 = f.read()  #接住
    f2 = ''.join(f1.split())  #把空白全去掉
    #print f2
    words=jieba.cut(f2)  #斷詞
    n = ' '.join(words)  #把斷詞結果用空白隔開
    n1 = n.encode('utf-8')  #編碼
    f.close()
    
    fid = open(filename,'a')
    
    #統計字頻
    dic={}
    for w in n1.split(' '):
        if w not in dic:
            dic[w] = 1
        else:
            dic[w] = dic[w] + 1
    import operator
    words_freq = sorted(dic.iteritems(),key=operator.itemgetter(1),reverse=True)
    #print attraction#,"\n總詞量：",len(words_freq)
    fid.write(attraction+',')
    fid.write('\n')
    #列出原始字頻排名
    #print "==========字頻統計=========="
    #print "排名","\t","詞彙","\t","字頻"
    #print "- - - - - - - - - - - - - - "
    num2 = 0
    for ele in words_freq[0:rankNum]:
        num2 += 1
        #print num2,"\t",ele[0],"\t",ele[1]
        #print ele[0]
        #print type(ele[0])
        #x = ele[0].encode('utf-8')
        fid.write(ele[0])        
        fid.write(',\n')
    fid.write('\n')
    fid.close()
#        if num2 % 10 == 0:
            #print "- - - - - - - - - - - - - - "
            
# 比對多個景點斷詞字頻的方法
def mutifrestat(date,attrlist,RankNum):
    filename = 'WordsFre-'+str(RankNum)+'-'+date+'.txt'  
    for i in range(0,len(attrlist)):
        jiebafrestat(attrlist[i],RankNum,filename)
        #print '\n'
########       
# 方法四：計算字頻 - 去除停用詞後的字頻統計
########
def stopWordsSet(stopwordfile):
    li = []
    fid =  open(stopwordfile,'r')
    for line in fid:
        li.append(line.strip())
    fid.close()
    setli = set(li)
    return setli

def jiebafrestopwords(attraction,stopwordfile,rankNum,filename):
#轉換象山資料,並切詞,並使用JIEBA內建的tfidf統計排名
    f = open(attraction,'r')  #讀取檔案
    f1 = f.read()  #接住
    f2 = ''.join(f1.split())  #把空白全去掉
    #print f2
    words=jieba.cut(f2)  #斷詞
    n = ' '.join(words)  #把斷詞結果用空白隔開
    n1 = n.encode('utf-8')  #編碼
    f.close()
    
    fid = open(filename,'a')
    
    #統計字頻
    dic={}
    comparelist = stopWordsSet(stopwordfile)
    for w in n1.split(' '):
        if w not in comparelist:
            if w not in dic:
                dic[w] = 1
            else:
                dic[w] = dic[w] + 1
    import operator
    words_freq = sorted(dic.iteritems(),key=operator.itemgetter(1),reverse=True)
    #print attraction#,"\n總詞量(去除停用詞後)：",len(words_freq)
    fid.write(attraction+',\n')
    #列出原始字頻排名
    #print "==========字頻統計=========="
    #print "排名","\t","詞彙","\t","字頻"
    #print "- - - - - - - - - - - - - - "
    num2 = 0
    for ele in words_freq[0:rankNum]:
        num2 += 1
        #print num2,"\t",ele[0],"\t",ele[1]
        #print ele[0]
        fid.write(ele[0])
        fid.write(',\n')
        #if num2 % 10 == 0:
            #print "- - - - - - - - - - - - - - "
    fid.write('\n')
    fid.close()        
# 比對多個景點斷詞字頻(去停用詞後)的方法
def mutifrestopwords(date,attrlist,stopWordsFile,RankNum):
    filename = 'WordsFreAddStop-'+str(RankNum)+'-'+date+'.txt' 
    for i in range(0,len(attrlist)):
        jiebafrestopwords(attrlist[i],stopWordsFile,RankNum,filename)
        #print '\n'

# 綜合比對
使用方法：

    1.新增一陣列：內含要分析景點的檔案名稱(TXT)
    2.決定要使用的停用詞字典(TXT)
    3.決定要顯示的排名數

使用範例：

    陣列 = [景點A檔,景點B檔,景點C檔]
    
    1.單一TFIDF
    --def mutijiebastat(date,attrlist,RankNum)
    mutijiebastat(日期,陣列,顯示排名數)
    
    2.多重TFIDF
    --def CompareTFIDF(date,listInput,stopWordsFile,rankNum)
    CompareTFIDF(日期,陣列,停用詞字典,顯示排名數)
    
    3.原始字頻統計
    --def def mutifrestat(date,attrlist,RankNum)
    mutifrestat(日期,陣列,顯示排名數)
    
    4.去停用詞後字頻統計
    --def mutifrestopwords(date,attrlist,stopWordsFile,RankNum)
    mutifrestopwords(日期,陣列,停用詞字典,顯示排名數)
    
實作：

    list = ['mountain.txt','lungshanTemple.txt','maokongGondola.txt']
    rank = 10
    date = '20151223'
    stpDic = 'stopwords_1222.txt'
    
    mutijiebastat(date,lis,rank)
    
    CompareTFIDF(date,lis,stpDic,rank)
    
    mutifrestat(date,lis,rank)
    
    mutifrestopwords(date,lis,stpDic,rank)

In [3]:
def FourWayPK(folder,date,rank,stpDic):
    #實作：
    import time
    import os
    lis = []
    for name in os.listdir(folder):
        path = folder+'\\'+name
        lis.append(path)

    x = time.time()
    #date = '20151223'
    #lis = ['mountain.txt','lungshanTemple.txt','maokongGondola.txt']
    #rank = 10
    #stpDic = 'stopwords_1227.txt'

    mutijiebastat(date,lis,rank)

    CompareTFIDF(date,lis,stpDic,rank)

    mutifrestat(date,lis,rank)

    mutifrestopwords(date,lis,stpDic,rank)

    y = time.time()
    print 'Finish!'
    print 'Cost ',y-x,' secs.'

# 綜合比對 方法
使用說明：

    FourWayPK(資料夾,日期,排名,停用詞字典)
    
    資料夾：[字串]。資料夾名稱，內含所有以景點為單位的TXT檔(命名要用英文)
           EX:有一'AttrSample'的資料夾，內有30個檔，
              每個檔都是一個單獨景點的所有評論\遊記。
              
    日　期：[字串]。輸入今天日期，格式-->'YYYYMMDD'
           EX:'20151223'
           
    排　名：[整數]。想要顯示各方法的前幾個排名數
           EX:10
           
    停用詞：[字串]。TXT檔。內容格式需為一行一個字詞，字詞後不得有逗點
           EX:'stopwords_1222.txt'

方法結果：

    會在所在的資料夾產生四個TXT檔，開頭分別為：
    1.JiebaTfidf　　　　-->單一TFIDF
    2.SciTfidf         -->多重TFIDF
    3.WordsFre         -->普通字頻統計
    4.WordsFreAddStop  -->去停用詞字頻統計
    
    分別是四個方法的結果，並且會依照景點排序。
    
    且會傳回所耗時數！
    
# 【注意】每次使用的＂日期＂都要設不一樣
# 　　　　否則會在同一個檔案一直append下去

In [4]:
FourWayPK('AttrSample','20151223',10,'stopwords_1227.txt')

Finish!
Cost  19.9559998512  secs.
